In [1]:
import pandas as pd
import glob
import datetime
import re
import warnings
import numpy as np
warnings.filterwarnings(action='ignore')

## 버스 데이터 파일 불러오기

In [2]:
file_list = glob.glob('./일반*.xlsx')
# file_list

In [3]:
# 정류장 번호
place = {
    '경주골굴사':['02089','02090'], '경주김유신묘':['01025','01036'], 
    '경주동궁과월지':['01237', '01238'], '경주동부사적지대':['01279','01280'], 
    '경주무열왕릉':['01194','01195'], '경주문무대왕릉':['02056','02057'],
    '경주불국사':['01160','01161'], '경주석굴암':['01199','01199'], 
    '경주신문왕릉':['01081','01082'], '경주월성(반월성)':['01237','01238'], 
    '경주첨성대':['01279','01280'], '경주금곡사':['05379','05380'],
    '경주기림사':['02021','02021'], '경주백률사':['01228','01229'],
    '경주분황사':['01157','01158'],'경주연지암':['05423','05426'], 
    '경주주사암':['03121','03122']
} 

In [4]:
bus_final = {'일반10': ['20', '21:40'],
             '일반11': ['20', '22:00'],
             '일반15': ['300', '18:30'],
             '일반16': ['100', '19:30'],
             '일반231': ['100', '19:20'],
             '일반234': ['100', '17:20'],
             '일반273': ['90', '20:00'],
             '일반30': ['80', '20:27'],
             '일반301': ['30', '19:15'],
             '일반31': ['80', '20:10'],
             '일반333': ['120', '18:25'],
             '일반334': ['85', '20:45'],
             '일반335': ['130', '18:40'],
             '일반337': ['120', '20:00'],
             '일반338': ['100', '19:25'],
             '일반40': ['27', '21:55'],
             '일반41': ['30', '21:55'],
             '일반50': ['20', '22:55'],
             '일반602': ['320', '18:30'],
             '일반604': ['155', '17:45'],
             '일반605': ['80', '20:00'],
             '일반607': ['100', '18:00'],
             '좌석10': ['20', '21:40'],
             '좌석100': ['20', '22:00'],
             '좌석100-1': ['20', '22:00'],
             '좌석11': ['20', '22:00'],
             '좌석12': ['60', '18:20'],
             '좌석130': ['60', '19:15'],
             '좌석150': ['50', '21:40'],
             '좌석150-1': ['50', '17:40'],
             '좌석18': ['115', '20:20'],
             '좌석300': ['30', '21:25'],
             '좌석303': ['170', '21:00'],
             '좌석332': ['100', '20:05'],
             '좌석350': ['30', '21:45'],
             '좌석601': ['105', '19:25'],
             '좌석700': ['60', '18:20'],
             '좌석710': ['60', '20:46'],
             '좌석711': ['60', '20:23']}

In [5]:
df = pd.read_excel(file_list[0])
del df['Unnamed: 0']

## '도착시간' 시간 계속 더해가기

In [6]:
time = int(df['도착시간'][1].split('분')[0])
sum_time = 0
for i in range(1, len(df['도착시간'])):
    time = int(df['도착시간'][i].split('분')[0])
    sum_time += time
    df['도착시간'].loc[i] = sum_time
df

,버스노선,정류장번호,버스번호,도착시간,배차간격
0,시외버스터미널,1224,일반231,06:25,평일 100분
1,보우주택,1148,일반231,1,평일 100분
2,청년회의소,1322,일반231,2,평일 100분
3,중앙시장,1307,일반231,3,평일 100분
4,법원.동산병원 건너,1083,일반231,4,평일 100분
5,신한은행사거리,1232,일반231,5,평일 100분
6,성동시장 동편,1028,일반231,7,평일 100분
7,세무서.큰마디병원,1211,일반231,8,평일 100분
8,경주교.신라중학교,1013,일반231,9,평일 100분
9,경주고용센타,1010,일반231,10,평일 100분


## 시간 변환 함수

In [7]:
def time_convert(time):
    hour = 0
    minute = 0
    second = 0
    if time >= 60:
        hour = time // 60
        minute = time % 60
    else:
        minute = time
    return [hour, minute]

## 배차 간격에 따른 컬럼 추가

In [8]:
# 운행 횟수
baecha_count = int(re.sub(r'[^0-9]','', df['배차간격'][0].split('분')[1])) - 1

# 배차간격
baecha_time = int(re.sub(r'[^0-9]','', df['배차간격'][0].split('분')[0]))
print(df['배차간격'][0], ' -> ', baecha_time, baecha_count)

ValueError: invalid literal for int() with base 10: ''

In [ ]:
# 배차간격 시간 변환 함수에 넣어서 시간 계산
baecha_time_sum = []
tmp = 0
for i in range(baecha_count):
    tmp += baecha_time
    baecha_time_sum.append(tmp)
baecha_time_sum

In [ ]:
for i in range(baecha_count):
    df[f'배차{i}'] = df['도착시간']
    time_test = time_convert(baecha_time_sum[i])
    df[f'배차{i}'][0] = str(int(df[f'배차{i}'][0].split(':')[0]) + time_test[0]) + ':' + str(int(df[f'배차{i}'][0].split(':')[1]) + time_test[1])
   
    for n in range(1, len(df)):
        df[f'배차{i}'][n] = df['도착시간'][n]
        a = time_convert(df['도착시간'][n])
        df[f'배차{i}'][n] = str(int(df[f'배차{i}'][0].split(':')[0]) + a[0]) + ':' + str(int(df[f'배차{i}'][0].split(':')[1]) + a[1])
        
        if int(df[f'배차{i}'][n].split(':')[1]) >= 60:
            b = time_convert(int(df[f'배차{i}'][n].split(':')[1]))
            df[f'배차{i}'][n] = str(int(df[f'배차{i}'][0].split(':')[0]) + b[0]) + ':' + str(b[1])
        
df

## 장소와 시간 입력에 따른 데이터 접근 

In [ ]:
# 장소 입력
print(list(place.keys()))
input_place = input('장소 입력: ')

In [ ]:
place[input_place]  # 유적지와 가장 가까운 정류장 번호

In [ ]:
# 시간 입력
input_hour, input_minute =  map(int, input().split())

## 유적지 지나는 버스 가져오기

In [ ]:
# 데이터프레임에 '정류장번호' string 변환 후 리스트에 저장
str_num = []
for x in df['정류장번호']:
    str_num.append(str(x).zfill(5))
str_num[:4]

In [ ]:
# 몇 번 버스 타야하는지 출력
time_data = []
name_data = []

# 입력한 유적지를 지나가는 버스 데이터 가져오기
for x in range(len(str_num)):
    for y in range(2):
        if str_num[x] == place[input_place][y]:
            print(df['버스번호'][0], df['버스노선'][x])
            name_data.append(df['버스노선'][x])

    # 입력한 시간 이후의 버스 도착시간 가져오기
            for z in range(baecha_count):
                hour = int(df[f'배차{z}'][x].split(':')[0])
                minute = int(df[f'배차{z}'][x].split(':')[1])
                if hour >= input_hour and minute >= input_minute:
                    time_data.append(df.loc[x][f'배차{z}'])

result = {'버스': df['버스번호'][0],
          '정류장':name_data,
          '시간표':time_data}
print(result)
    

## 한 번에 가는 코스
* n개의 코스 입력
* 두 유적지가 있는 버스 데이터를 가져옴
 - 조건 1. 먼저 입력했던 유적지가 앞에 있어야 함
 - 조건 2. 두 번째 유적지부터 관람시간(1시간)을 더한 상태로 계산
* 조건 1, 2를 반복하면서 n-1개의 버스를 구함

* 모든 버스 데이터 불러오기
* 모든 데이터프레임에 배차간격 컬럼 추가
* 

In [ ]:
# 모든 버스데이터 불러오기
file_list = glob.glob('./data/*.xlsx')

In [ ]:
for i in range(len(file_list)):
    
    # 리스트에 모든 버스 데이터프레임 넣기
    df = pd.read_excel(file_list[i])
    del df['Unnamed: 0']
    df_list.append(df)

In [ ]:
# 모든 데이터프레임에 배차간격 컬럼 추가
df_list = []
for i in range(len(file_list)):
    
    # 리스트에 모든 버스 데이터프레임 넣기
    df = pd.read_excel(file_list[i])
    del df['Unnamed: 0']
    df_list.append(df)
    
    #### 각 데이터프레임에 배차간격 컬럼 추가
    # 운행 횟수
    baecha_count = int(re.sub(r'[^0-9]','', df_list[i]['배차간격'][0].split('분')[1])) - 1
    # 배차간격
    baecha_time = int(re.sub(r'[^0-9]','', df_list[i]['배차간격'][0].split('분')[0]))
    
    # 배차간격 시간 변환 함수에 넣어서 시간 계산
    baecha_time_sum = []
    tmp = 0
    for x in range(baecha_count):
        tmp += baecha_time
        baecha_time_sum.append(tmp)
    
    df_list[i]

* 평일(매일) {}분 {}회
* 평일(매일) {}분 
* 평일(매일) {}회
* 평일(매일) {}-{}분 {}회

In [ ]:
bus_final_list = list(bus_final.items())
bus_final_list[0][0]

In [ ]:
for i in range(len(bus_final_list)):
    for j in range(len(df_list)):
        if bus_final_list[i][0] == df_list[j]['버스번호'][0]:
            df_list[j]['배차간격'] = bus_final_list[i][1][0]
            df_list[j]['막차'] = bus_final_list[i][1][1]
df_list[0]

In [ ]:
lst = []
for x in df_list:
    lst.append(x['버스번호'][0])
set(lst)

In [ ]:
df_list

In [ ]:
input_count = int(input("유적지 개수 입력: "))

# input_hour, input_minute =  map(int, input(f"{i}번 째 유적지 입력: ").split())

for i in range(input_count):
    print(list(place.keys()), '\n')
    input_place = input(f"{i}번 째 유적지 입력: ")
    

## 환승 함수 구하기

* 모든 버스 데이터에 접근해서 중복되는 정류장 번호를 찾음
 - 조건 1. 처음 버스는 중복 정류장보다 앞에 있어야 함
 - 조건 2. 다음 버스는 중복 정류장보다 뒤에 있어야 함

In [ ]:
n 

## 모든 버스 데이터 가져와서 for문으로 돌리는 코드